In [ ]:
!pip uninstall neo4j -y
!pip install neo4j

Found existing installation: neo4j 5.14.0
Uninstalling neo4j-5.14.0:
  Successfully uninstalled neo4j-5.14.0
  Using cached neo4j-5.28.2-py3-none-any.whl.metadata (5.9 kB)
Using cached neo4j-5.28.2-py3-none-any.whl (313 kB)


In [ ]:
!pip install langchain

In [ ]:
pip install langchain-community

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install langchain-groq

# Setting Environment Variables

In [ ]:
NEO4J_URI="neo4j+s://79af7187.databases.neo4j.io"
NEO4J_USERNAME="neo4j"

In [ ]:
import os
from google.colab import userdata
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = userdata.get('NEO4J_PASSWORD')

# Initalize GraphDb

In [ ]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=os.environ["NEO4J_URI"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    database="neo4j" # Specify the database name
)
graph

/tmp/ipython-input-1584867546.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


# Cypher script

In [ ]:
# Movie Dataset
movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/5e3939d10216b7663687217c1646c30eb921d92f/movies/movies_small.csv' AS row

MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director IN split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m)
)
FOREACH (actor IN split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)
)
FOREACH (genre IN split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
)
"""

In [ ]:
movie_query

"\nLOAD CSV WITH HEADERS FROM \n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/5e3939d10216b7663687217c1646c30eb921d92f/movies/movies_small.csv' AS row\n\nMERGE (m:Movie {id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director IN split(row.director, '|') |\n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m)\n)\nFOREACH (actor IN split(row.actors, '|') |\n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m)\n)\nFOREACH (genre IN split(row.genres, '|') |\n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g)\n)\n"

# Load the data into GraphDb

In [ ]:
graph.query(movie_query)

[]

In [ ]:
graph.refresh_schema()

In [ ]:
print(graph.schema)

Node properties:
PERSON {name: STRING, born: STRING}
Movie {title: STRING, released: STRING, id: STRING, imdbRating: FLOAT}
User {name: STRING, city: STRING, userId: INTEGER, age: INTEGER}
POST {postId: INTEGER, content: STRING, timestamp: DATE_TIME}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:PERSON)-[:ACTED_IN]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)
(:User)-[:Posted]->(:POST)
(:User)-[:FRIEND]->(:User)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


# Load LLM model

In [ ]:
groq_api_key = userdata.get('GROQ_API_KEY')

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a1791606930>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a17913867b0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

# GraphQuery Chain with LLM

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7a17ae9bc200>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a1791606930>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a17913867b0>, model_name='Gemma2-9b-I

# Retrieve answers from GraphDb using GraphQuery Chain

In [ ]:
response=chain.invoke("Which movies were directed by Peter Jackson?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name:"Peter Jackson"})-[directed]->(m:Movie)
RETURN m.title

Full Context:
[{'m.title': 'Heavenly Creatures'}]

> Finished chain.


{'query': 'Which movies were directed by Peter Jackson?',
 'result': 'Heavenly Creatures \n'}

In [ ]:
response=chain.invoke("Tell me which genres films were most popular?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WITH g, COUNT(m) AS genrePopularity
ORDER BY genrePopularity DESC
RETURN g.name, genrePopularity 

Full Context:
[{'g.name': 'Drama', 'genrePopularity': 162}, {'g.name': 'Comedy', 'genrePopularity': 104}, {'g.name': 'Romance', 'genrePopularity': 63}, {'g.name': 'Thriller', 'genrePopularity': 55}, {'g.name': 'Action', 'genrePopularity': 46}, {'g.name': 'Crime', 'genrePopularity': 40}, {'g.name': 'Adventure', 'genrePopularity': 31}, {'g.name': 'Children', 'genrePopularity': 28}, {'g.name': 'Sci-Fi', 'genrePopularity': 20}, {'g.name': 'Horror', 'genrePopularity': 17}]

> Finished chain.


{'query': 'Tell me which genres films were most popular?',
 'result': 'Drama was the most popular genre, followed by Comedy.  \n'}

In [ ]:
response=chain.invoke("Suggest me a good Romance genres movie ?")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre {name: "Romance"}) RETURN m.title
Full Context:
[{'m.title': 'Grumpier Old Men'}, {'m.title': 'Waiting to Exhale'}, {'m.title': 'Sabrina'}, {'m.title': 'American President, The'}, {'m.title': 'Cutthroat Island'}, {'m.title': 'Sense and Sensibility'}, {'m.title': 'Leaving Las Vegas'}, {'m.title': 'Persuasion'}, {'m.title': 'Carrington'}, {'m.title': 'Clueless'}]

> Finished chain.


{'query': 'Suggest me a good Romance genres movie ?',
 'result': 'Waiting to Exhale, Sabrina,  American President, The,  Sense and Sensibility or Persuasion would be good choices. \n'}

In [ ]:
response=chain.invoke(" which directors directed most movies ? ")
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie)
WITH p, COUNT(m) as movieCount
ORDER BY movieCount DESC
RETURN p.name, movieCount LIMIT 10 

Full Context:
[{'p.name': 'Krzysztof Kieślowski', 'movieCount': 3}, {'p.name': 'Robert Rodriguez', 'movieCount': 3}, {'p.name': 'John N. Smith', 'movieCount': 2}, {'p.name': 'Allison Anders', 'movieCount': 2}, {'p.name': 'Mike Figgis', 'movieCount': 2}, {'p.name': 'Yimou Zhang', 'movieCount': 2}, {'p.name': 'Martin Scorsese', 'movieCount': 2}, {'p.name': 'Oliver Stone', 'movieCount': 2}, {'p.name': 'Ang Lee', 'movieCount': 2}, {'p.name': 'Quentin Tarantino', 'movieCount': 2}]

> Finished chain.


{'query': ' which directors directed most movies ? ',
 'result': 'Krzysztof Kieślowski and Robert Rodriguez directed the most movies.  \n'}